# Pandas

The following is a quick note on how to transfer data extracted via datachef into a pandas dataframe.

| <span style="color:green">Note - pandas is a huge and ever evolving ecosphere all of itself. As such we're not going to bring it into datachef as a dependency (it has very differnet goals and we like our simple dependency chain). We do however provide the following conveniences for quickly shunting tidy data outputs into pandas in a suitably decoupled way</span>|
|-----------------------------------------|

## Source Data

The data source we're using for these examples is shown below:

The [full data source can be viewed here](https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/csv/bands-wide.csv).

In [ ]:
from datachef import acquire, preview
from datachef.selection import CsvSelectable

table: CsvSelectable = acquire.csv.http("https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/csv/bands-wide.csv")
preview(table)

## TidyData.to_dict()

The following is an example of using the `TidyData.from_dict()` method.

Note - the dict in question is the same structure used by the pandas `DataFrame.from_dict()` method - this will form our mechanism of handover.

In [ ]:
import json
from datachef import acquire, preview, filters
from datachef.direction import right, below
from datachef.selection import CsvSelectable
from datachef.output import TidyData, Column

table: CsvSelectable = acquire.csv.http("https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/csv/bands-wide.csv")

observations = table.filter(filters.is_numeric).label_as("Observation")
bands = (table.excel_ref("A3") | table.excel_ref("G3")).label_as("Band")
assets = table.excel_ref('2').is_not_blank().label_as("Asset")
members = (table.excel_ref("B") | table.excel_ref("H")).is_not_blank().label_as("Member")
preview(observations, bands, assets, members)

tidy_data = TidyData(
    observations,
    Column(bands.finds_observations_closest(right)),
    Column(assets.finds_observations_directly(below)),
    Column(members.finds_observations_directly(right))
)

# See the non dict tidydata
print(tidy_data)

# Now see it in dictionary form.
# We'll use json to prettyify the dict version
print(json.dumps(tidy_data.to_dict(), indent=2))

A dictionary like the above can be passed directly into pandas trivially as per the following

```
import pandas as pd

# ..
# ...
# the code from the above example
# ...
# ..


tidy_data_dict = tidy_data.to_dict()
dataframe = pd.DataFrame.from_dict(tidy_data_dict)
```